In [1]:
# standard library
import sys

# pydata stack
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import svm

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
print(f'Python version: {sys.version}')
print(f'pandas version: {pd.__version__}')
print(f'seaborn version: {sns.__version__}')
print(f'scikit-learn: {sklearn.__version__}')

---

## Load Dataset

In [ ]:
iris = pd.read_csv('data/iris.csv')

---

## Explore with Pandas

In [ ]:
iris.head()

In [ ]:
iris.info()

In [ ]:
iris['species'] = iris['species'].astype('category')

In [ ]:
iris.info()

In [ ]:
values = ['count', 'min', 'max', 'mean', 'std']
iris.groupby(by='species').agg(values)

---

## Visualization

In [ ]:
iris.plot.hist()

In [ ]:
iris['sepal_length'].plot.hist(by='species')

In [ ]:
iris['sepal_width'].plot.hist()

In [ ]:
iris['petal_length'].plot.hist()

In [ ]:
iris['petal_width'].plot.hist()

---

In [ ]:
iris.boxplot(by='species', figsize=(20, 15))

---

## Multidimensional Plots

In [ ]:
from pandas.plotting import scatter_matrix

In [ ]:
scatter_matrix(iris, figsize=(12, 12), diagonal='kde')

In [ ]:
from pandas.plotting import andrews_curves

In [ ]:
plt.figure(figsize=(10, 10))
andrews_curves(iris, 'species')

In [ ]:
from pandas.plotting import parallel_coordinates

In [ ]:
plt.figure(figsize=(10, 10))
parallel_coordinates(iris, 'species')

In [ ]:
from pandas.plotting import radviz

In [ ]:
plt.figure(figsize=(10, 10))
radviz(iris, 'species')

---

## Seaborn Visualization

In [ ]:
sns.set(color_codes=True)

### Distributions

In [ ]:
sns.distplot(iris['sepal_width'])

In [ ]:
sns.distplot(iris['sepal_width'],
             hist=True,
             kde=True,
             rug=True,
             bins=5)

In [ ]:
# Kernel Density Estimate... can specify the kind f kernel
sns.kdeplot(iris['sepal_width'],
            shade=True,
            kernel='gau',
            bw=.13)

In [ ]:
from scipy import stats

In [ ]:
sns.distplot(iris['petal_width'],
             hist=True,
             kde=True,
             fit=stats.expon) ## pass in random variable object

In [ ]:
# Jointplot (bivariate distributions)
sns.jointplot(data=iris,
              x='sepal_length',
              y='sepal_width',
              kind='scatter')

In [ ]:
# pairwise relationship
sns.pairplot(iris,
             hue='species')

In [ ]:
g = sns.PairGrid(iris)
g.map_diag(sns.kdeplot, shade=True)
g.map_offdiag(sns.kdeplot, cmap="Blues_r", n_levels=5)

### Categorical Data

In [ ]:
iris['above_sepal_width_mean'] = iris['sepal_width'] > iris['sepal_width'].mean()

#### Scatterplot

In [ ]:
sns.stripplot(data=iris,
              x='species',
              y='sepal_width',
              hue='above_sepal_width_mean',
              jitter=True)

#### Distributions

In [ ]:
sns.violinplot(data=iris,
               x='species',
               y='sepal_width',
               hue='above_sepal_width_mean',
               inner='box')

---

## Build and plot Support Vector Machine classifers

Adapted from [sklearn docs](http://scikit-learn.org/stable/auto_examples/svm/plot_iris.html)

In [ ]:
# convert to sklearn format, grab first 2 columns
X = np.array(iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])[:, :2]

# convert strings to categorical
le = sklearn.preprocessing.LabelEncoder()
le.fit(iris['species'])
y = le.transform(iris['species'])

In [ ]:
h = .02  # step size in the mesh

# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
svc = sklearn.svm.SVC(kernel='linear', C=C).fit(X, y)
rbf_svc = sklearn.svm.SVC(kernel='rbf', gamma=0.7, C=C).fit(X, y)
poly_svc = sklearn.svm.SVC(kernel='poly', degree=3, C=C).fit(X, y)
lin_svc = sklearn.svm.LinearSVC(C=C).fit(X, y)

# create a mesh to plot in
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# title for the plots
titles = ['SVC with linear kernel',
          'LinearSVC (linear kernel)',
          'SVC with RBF kernel',
          'SVC with polynomial (degree 3) kernel']


for i, clf in enumerate((svc, lin_svc, rbf_svc, poly_svc)):
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    plt.subplot(2, 2, i + 1)
    plt.subplots_adjust(wspace=0.4, hspace=0.4)

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm)
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())
    plt.title(titles[i])

plt.show()